# Basic Example: Prompt+Model+OutputParser



## Overview

最基本最常見的用法是將提示模板與模型連結起來。為了說明如何使用，我們將建立一個鏈，要求提供各種國家的首都。


### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [Utilizing Prompt Templates](#utilizing-prompt-templates)
- [Chain Creation](#chain-creation)

### References

- [LangChain ChatOpenAI API reference](https://python.langchain.com/api_reference/openai/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html)
- [LangChain Core Output Parsers](https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.list.CommaSeparatedListOutputParser.html#)
- [Python List Tutorial](https://docs.python.org/3.13/tutorial/datastructures.html)
---

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- ```langchain-opentutorial``` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [```langchain-opentutorial```](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [1]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_openai",
        "langchain_community",
    ],
    verbose=False,
    upgrade=False,
)

In [2]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "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",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "",
    }
)

Environment variables have been set successfully.


---
你也可以在 ```.env``` 文件中設定 ```OPENAI_API_KEY```，並且載入它。 

[Note] 如果你已經在之前的步驟中设置了 ```OPENAI_API_KEY```，那麽這一步是不必要的。

In [4]:
# Configuration File for Managing API Key as an Environment Variable
from dotenv import load_dotenv

# Load API KEY Information
load_dotenv(override=True)

True

In [5]:
# Set up LangSmith tracking: https://smith.langchain.com
from langsmith import utils

utils.tracing_is_enabled()

True

## Utilizing Prompt Templates

```PromptTemplate```

- PromptTemplate常用來 創造一個完整的提示字串，其中包含使用者的輸入變數。
- Usage
  - ```template```: 模板字串是一個預先定義的格式，其中使用大括號 '{}' 來表示變數。

  - ```input_variables```: 定義了要插入大括號中的變數名稱的清單。

```input_variables```

- ```input_variables```定義了要插入大括號中的變數名稱的清單。

In [6]:
from langchain_core.prompts import PromptTemplate

The ```from_template()``` method is used to create a ```PromptTemplate``` object.

In [7]:
# Define template
template = "What is the capital of {country}?"

# Create a `PromptTemplate` object using the `from_template` method.
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='What is the capital of {country}?')

In [8]:
# Generate the prompt.
prompt = prompt_template.format(country="Korea")
prompt

'What is the capital of Korea?'

In [9]:
# Generate the prompt.
prompt = prompt_template.format(country="USA")
prompt

'What is the capital of USA?'

In [10]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

## Chain Creation

### LCEL (LangChain Expression Language)

這裏, 我們使用 LCEL 將各種組件組合成一個單一的鏈.

![lcel.png](./assets/02-langchain-expression-language.png)

```
chain = prompt | model | output_parser
```

```|``` 符號類似於 [Unix pipe operator](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>), 連結不同的組件並傳遞一個組件的輸出作為下一個組件的輸入.

在這個鏈中, 使用者輸入被傳遞到提示模板, 並從提示模板輸出轉發到模型. 通過獨立檢查每個組件, 你可以理解每個步驟發生什麼.

In [11]:
# Create the prompt as a `PromptTemplate` object.
prompt = PromptTemplate.from_template("Please explain {topic} in simple terms.")


# Combine the prompt and model into a chain
chain = prompt | model

### Calling ```invoke()```

- 輸入值以 Python 字典 (key-value pairs) 的形式提供.  
- 在呼叫 ```invoke()``` 函數時, 這些輸入值會作為參數傳遞.

In [ ]:
# 設定   topic 在 `input` 字典中為 'The Principles of Learning in Artificial Intelligence Models'.
input = {"topic": "The Principles of Learning in Artificial Intelligence Models"}

In [ ]:
# 將 `prompt` 物件和 `model` 物件使用管道 (`|`) 運算子連接.
# 使用 `invoke` 方法傳遞 `input`.
# 這將返回 AI 模型生成的消息.
chain.invoke(input)

AIMessage(content='Sure! The principles of learning in artificial intelligence (AI) models can be understood as the basic ideas that guide how these models learn from data. Here are some key principles explained in simple terms:\n\n1. **Data is Key**: AI models learn from data. The more relevant and high-quality data they have, the better they can learn and make predictions. Think of it like a student learning from textbooks; the better the books, the more they learn.\n\n2. **Patterns and Features**: AI looks for patterns in the data. It identifies important features (characteristics) that help it understand the information. For example, if it’s learning to recognize cats in pictures, it might focus on features like fur texture, ear shape, and eye color.\n\n3. **Training and Testing**: AI models go through a training phase where they learn from a set of data. After training, they are tested on new data to see how well they learned. This is like practicing for a test and then taking the

下面是一個輸出流式回應的示例:

In [14]:
# Request for Streaming Output
answer = chain.stream(input)

# Streaming Output
for token in answer:
    print(token.content, end="", flush=True)

Sure! The Principles of Learning in Artificial Intelligence (AI) Models can be understood as the basic ideas that guide how AI systems learn from data and improve their performance over time. Here are some key principles explained in simple terms:

1. **Data is Key**: AI models learn from data. The more quality data they have, the better they can learn. Think of it like a student studying for a test; the more information they have, the better they can do.

2. **Learning from Examples**: AI models learn by looking at examples. For instance, if you show an AI many pictures of cats and dogs, it can learn to tell the difference between them. This is similar to how humans learn by observing and practicing.

3. **Feedback Loop**: AI models improve through feedback. When they make mistakes, they can adjust their understanding based on the corrections. This is like a teacher giving feedback to a student to help them learn from their errors.

4. **Generalization**: AI models aim to generalize f

### Output Parser

一個 **Output Parser** 是一個工具, 用於將 AI 模型的回應轉換或處理成特定格式. 由於模型的輸出通常以自由格式文本提供, 一個 **Output Parser** 是必要的, 用於將其轉換為結構化格式或提取所需數據.


In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = (
    StrOutputParser()
)  # 直接返回模型的回應作為字符串, 不進行修改.

An output parser is added to the chain.

In [16]:
# A processing chain is constructed by connecting the prompt, model, and output parser.
chain = prompt | model | output_parser

In [17]:
# Use the invoke method of the chain object to pass the input
chain.invoke(input)

"Sure! The Principles of Learning in Artificial Intelligence (AI) Models can be understood as the basic ideas that guide how AI systems learn from data and improve their performance over time. Here are some key principles explained in simple terms:\n\n1. **Data is Key**: AI models learn from data. The more relevant and high-quality data they have, the better they can learn. Think of it like a student studying for a test; the more information they have, the better they can prepare.\n\n2. **Learning from Examples**: AI models often learn by looking at examples. For instance, if you want to teach an AI to recognize cats in pictures, you show it many pictures of cats and non-cats. The model learns to identify patterns that distinguish cats from other objects.\n\n3. **Feedback Loop**: AI models improve through feedback. After making predictions or decisions, they receive feedback on whether they were right or wrong. This feedback helps them adjust and learn from their mistakes, similar to h

In [18]:
# Request for Streaming Output
answer = chain.stream(input)

# Streaming Output
for token in answer:
    print(token, end="", flush=True)

Sure! The Principles of Learning in Artificial Intelligence (AI) Models can be understood as the basic ideas that guide how AI systems learn from data and improve their performance over time. Here are some key principles explained in simple terms:

1. **Data is Key**: AI models learn from data. The more quality data they have, the better they can learn. Think of it like a student studying for a test; the more information they have, the better they can do.

2. **Learning from Examples**: AI models often learn by looking at examples. For instance, if you want an AI to recognize cats in pictures, you show it many pictures of cats and non-cats. Over time, it learns to tell the difference.

3. **Feedback Loop**: AI models improve through feedback. When they make mistakes, they can adjust their understanding based on the corrections. This is similar to how a teacher helps a student learn from their errors.

4. **Generalization**: A good AI model can apply what it has learned to new, unseen d

### Applying and Modifying Templates

- The prompt content below can be **modified** as needed for testing purposes.  
- The ```model_name``` can also be adjusted for testing.

In [ ]:
template = """
你是有 10 年經驗的英文老師, 請寫一個適合給定情況的英文對話.  
請參考 [FORMAT] 的結構.

# 情況:
{question}

# 格式:
- 英文對話:
- 英文對話的解釋: 
"""

# 產生 prompt
prompt = PromptTemplate.from_template(template)

# 初始化 ChatOpenAI 模型
model = ChatOpenAI(model_name="gpt-4o-mini")

# 初始化 string output parser
output_parser = StrOutputParser()

In [20]:
# Construct the chain.
chain = prompt | model | output_parser

In [ ]:
# Execute the completed Chain to obtain a response.
print(chain.invoke({"question": "我想要去餐廳訂餐."}))

- Dialogue in English:
**Waiter:** Good evening! Welcome to La Bella Italia. How many are in your party?  
**Customer:** Just one, please.  
**Waiter:** Right this way. Here’s your menu. Can I get you something to drink while you look?  
**Customer:** Yes, I’ll have a glass of water, please.  
**Waiter:** Sure! Are you ready to order, or do you need more time?  
**Customer:** I think I’m ready. I’d like the spaghetti carbonara, please.  
**Waiter:** Excellent choice! Would you like any appetizers or desserts with that?  
**Customer:** I’ll have a side salad to start, and maybe a slice of tiramisu for dessert.  
**Waiter:** Great! I’ll put that order in for you. Anything else I can get you?  
**Customer:** No, that’s all for now, thank you.  
**Waiter:** You’re welcome! I’ll be back shortly with your order.  

- Explanation of the Dialogue: 
In this conversation, the customer arrives at a restaurant and interacts with the waiter. The waiter greets the customer and asks how many people a

In [ ]:
# Execute the completed Chain to obtain a response
# Request for Streaming Output
answer = chain.stream({"question": "我想要去餐廳訂餐."})

# Streaming Output
for token in answer:
    print(token, end="", flush=True)

- Dialogue in English:

**Customer:** Hi there! Can I get a table for two, please?  

**Host:** Sure! Right this way. Here’s your menu.  

**Customer:** Thank you! What do you recommend for a starter?  

**Host:** Our bruschetta is very popular, and the garlic shrimp is a favorite as well.  

**Customer:** Sounds delicious! We'll have the bruschetta to start.  

**Host:** Great choice! And for the main course?  

**Customer:** I’d like the grilled salmon, please. How about you?  

**Friend:** I’ll have the steak, medium rare.  

**Host:** Excellent choices! Would you like to add any sides?  

**Customer:** Yes, we’ll take a side of roasted vegetables and a Caesar salad.  

**Host:** Perfect! I’ll get that order started for you.  

**Customer:** Thank you!  

**Host:** You’re welcome! Enjoy your meal.  

---

- Explanation of the Dialogue: 

In this dialogue, the customer arrives at a restaurant and requests a table for two. The host guides them to their table and hands them the menu. T

In [ ]:
# This time, set the question to 'Ordering Pizza in the US' and execute it.
# Request for Streaming Output
answer = chain.stream({"question": "在美國訂購比薩"})

# Streaming Output
for token in answer:
    print(token, end="", flush=True)

- Dialogue in English:

**Customer:** Hi there! I’d like to order a pizza, please.

**Pizza Server:** Of course! What size would you like? We have small, medium, large, and extra-large.

**Customer:** I’ll take a large, please. 

**Pizza Server:** Great choice! What type of pizza do you want? We have pepperoni, cheese, veggie, and a few specialty pizzas.

**Customer:** I’ll go with pepperoni. Can I add extra cheese?

**Pizza Server:** Absolutely! Extra cheese on a large pepperoni pizza. Would you like anything else? 

**Customer:** Yes, can I also get a side of garlic bread and a two-liter soda?

**Pizza Server:** Sure! We have a few soda options. We have cola, diet cola, lemon-lime, and root beer. Which one would you like?

**Customer:** I’ll take a cola, please.

**Pizza Server:** Great! So that’s one large pepperoni pizza with extra cheese, a side of garlic bread, and a cola. Would you like to add any dipping sauces?

**Customer:** Yes, please! Can I get a marinara sauce and a ranch